#### BackPropagation
Carlos Garcia - 21000475

<img src="\imgs\red.png">

In [2]:
import numpy as np

In [5]:
x = np.array([[3, 10]], dtype = 'float')

In [21]:
h3 = np.array([
               [-0.23, 0.22, 0.77],
               [0.9, 0.88, -0.4]
            ])


h2 = np.array([
               [-0.3, 0.62, 0.45],
               [0.45, 0.57, 0.48],
               [0.65, 0.58, -0.45]
            ])

h1 = np.array([
               [0.3, 0.46, 0.02],
               [0.22, -0.7, 0.65],
               [0.65, 0.9, 0.34]
            ])

In [22]:
delta_h3 = np.matmul(x, h3)
delta_h2 = np.matmul(delta_h3, h2)
delta_h1 = np.matmul(delta_h2, h1)

In [23]:
layers = [delta_h3, delta_h2, delta_h1]
i = 3
for layer in layers:
    n = np.argmax(layer.flatten())
    print('\nHidden Layer {} ---------------'.format(i))
    print(layer)
    print('Neuron with most error is: {}'.format(n+1))
    i -= 1


Hidden Layer 3 ---------------
[[ 8.31  9.46 -1.69]]
Neuron with most error is: 2

Hidden Layer 2 ---------------
[[0.6655 9.5642 9.0408]]
Neuron with most error is: 2

Hidden Layer 1 ---------------
[[8.180294 1.74791  9.303912]]
Neuron with most error is: 3
